# Force Calculator

Given a set of parameters, this notebook calculates the required amperage in the wire to flip the magnet.

The following contains a diagram of the setup.

![setup diagram](diagrams/setup.png)

## Imports

In [145]:
from collections import ChainMap
import sympy as sym
import sympy.vector as vec
from scipy import constants

## Parameters

### Parameters

In [299]:
# Solenoid Parameters
solenoid_radius = 0.02 # m
wire_radius = 0.000792 # m
number_of_levels = 1
turns_per_level = 1 # how many coils are at the same level
amperage = 0.04 # A, max is 5A

# Dipole Position Parameters
dipole_z = 0.001 # m
dipole_x = 0 # m, in the direction of r_hat where x = 0 is the center of the solenoid

# Dipole Properties
dipole_moment_magnitude = 0.0168 # A m^2. Use https://www.kjmagnetics.com/calculator.asp to calculate this.

## Magnetic Field Equation

In [88]:
C = vec.CoordSys3D('C', transformation='cylindrical', vector_names=list('rtk'), variable_names=list('RTK'))

(
D, # Radius of the solenoid
r_0, # Distance along theta = 0 from the center of the solenoid
z_0, # Distance along z from the center of the solenoid
theta, # Angle from the center of the solenoid
pi, # Pi
mu_0, # Permeability of free space
) = sym.symbols('D r_0 z_0 theta pi mu_0', real=True, positive=True)

I = sym.symbols('I', real=True) # Current in the solenoid

# Magnetic field of a solenoid
B_original = mu_0*I*D / (4 * pi) * sym.integrate((z_0 * C.r + (D - r_0) * C.k) / (D**2 - 2*r_0*D*sym.cos(theta) + r_0**2 + z_0**2)**(3/2), (theta, 0, 2*pi))
B_point = B_original.subs({ mu_0: constants.mu_0, pi: constants.pi })

Testing...

In [281]:
# Compare to the answer 6.2831 * 10^-7 k_hat
B_point.subs({ r_0: 0, z_0: 0, D: 0.001, I: 10 }).simplify()

0.0062831853106*C.k

Check whether we can estimate distance

In [290]:
print(B_point.subs({ r_0: 0, z_0: 0, D: 0.03, I: 5 }).simplify().magnitude())
print(B_point.subs({ r_0: 0, z_0: 0, D: 0.03+0.000792, I: 5 }).simplify().magnitude())
print(B_point.subs({ r_0: 0, z_0: 100*0.000792, D: 0.03+4*0.000792, I: 5 }).simplify().magnitude())

0.000104719755176667
0.000102026261863471
1.41331602490524e-5


### Use Superposition to Extend to Solenoid

In [207]:
Z = sym.IndexedBase('Z')
R = sym.IndexedBase('R')

i, j = sym.symbols('i j', cls=sym.Idx)
M, N = sym.symbols('M N', integer=True, positive=True)
B_indexed = B_point.subs({ z_0: Z[i], D: R[j] })
# Where N is the number of turns per level and M is the number of levels
B_sum = sym.Sum(sym.Sum(B_indexed, (j, 0, N)), (i, 0, M))

# Preview the sum
B_sum.simplify()

1.00000000054438e-7*I*Sum((C.k*(-r_0 + R[j]) + C.r*Z[i])*R[j]*Integral((r_0**2 - 2*r_0*cos(theta)*R[j] + R[j]**2 + Z[i]**2)**(-1.5), (theta, 0, 6.28318530717959)), (j, 0, N), (i, 0, M))

Plugging in values!

In [302]:
Z_vals = [C.K + wire_radius * i for i in range(number_of_levels)]
R_vals = [solenoid_radius + wire_radius * j for j in range(turns_per_level)]

B = B_sum.subs({
    N: turns_per_level - 1,
    M: number_of_levels - 1,
    I: amperage,
    r_0: C.R,
}).doit().subs({
    Z[i]: sym.simplify(Z_vals[i]) for i in range(number_of_levels)
}).subs({
    R[j]: sym.simplify(R_vals[j]) for j in range(turns_per_level)
})

## Force Acting on Dipole

In [303]:
# Magnetic moment of the dipole, as a vector
mu = - dipole_moment_magnitude * C.k
delop = vec.Del()

# Convert B to a Vector type
F_dipole = mu.dot(delop)(B).subs({
    C.R: dipole_x,
    C.K: dipole_z,
})

F_dipole.simplify()

(-1.04376151040477e-6)*C.r + (1.57350981468055e-7)*C.k